In [19]:
from core.goodreads import GoodReadsData
import gc

goodreads = GoodReadsData()
filename = goodreads.file_names[5]
filename

'goodreads_reviews_dedup'

In [20]:
gc.collect()

11

# Análisis de `goodreads_reviews_dedup`

In [21]:
import os
import numpy as np
import pandas as pd

In [22]:
# Download
if not os.path.exists(goodreads.get_file_path(filename)):
    goodreads.download_file(filename)
    
goodreads.count_samples(filename)

15739967

In [23]:
# Load
df = goodreads.load_file_range(filename, (0, 99999))

### Análisis
1. Comprensión de datos.
2. Detección de valores `nan` ocultos.
3. Detección de errores o incongruencias.
4. Detección de duplicados

In [24]:
df.sample(5)

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
30188,246eac0ca4d02e73d8768a5acb9b89d7,562933,4eb22d1d5cd709bd3e8e1385546c8d0c,2,We used 6 of the Arch Book series for Easter c...,Tue Feb 07 10:33:27 -0800 2017,Fri Jul 14 16:46:10 -0700 2017,,Tue Feb 07 00:00:00 -0800 2017,0,0
31706,037c382333b7084ccee4cdec7401de90,18584807,0ef0105953832438235ed568659c5c39,3,Lindsey Sommerfield lost her parents at an ear...,Sat May 03 11:30:45 -0700 2014,Sun Apr 05 16:00:12 -0700 2015,Mon May 19 00:00:00 -0700 2014,Mon May 19 00:00:00 -0700 2014,0,0
70460,b56a79469299b9dd55288709dfc9c46c,13344505,4f47703ad21e0be56f3f0e5e196c774c,5,"Okay, I cried in rounds on this book. If you'v...",Fri Sep 14 14:34:50 -0700 2012,Tue Sep 18 16:18:40 -0700 2012,Mon Sep 17 00:00:00 -0700 2012,Fri Sep 14 00:00:00 -0700 2012,0,1
84515,b8f6bbbd16b9f5dc8b3eea199386d0d2,16299,d3a1648030164510741be0d4f882197c,3,I tried this out - since a friend gave it a 5 ...,Mon Oct 07 10:13:23 -0700 2013,Mon Oct 07 10:16:31 -0700 2013,Mon Oct 07 10:16:31 -0700 2013,Mon Oct 07 00:00:00 -0700 2013,0,0
89120,37b3e60b4e4152c580fd798d405150ff,6742166,9a48b296080dca2b4e58ff457216a362,4,The first issue is phenomenal. The rest is a g...,Thu Jul 11 15:19:09 -0700 2013,Thu Jul 11 15:19:43 -0700 2013,Sun Jul 17 00:00:00 -0700 2011,,0,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       100000 non-null  object
 1   book_id       100000 non-null  object
 2   review_id     100000 non-null  object
 3   rating        100000 non-null  int64 
 4   review_text   100000 non-null  object
 5   date_added    100000 non-null  object
 6   date_updated  100000 non-null  object
 7   read_at       100000 non-null  object
 8   started_at    100000 non-null  object
 9   n_votes       100000 non-null  int64 
 10  n_comments    100000 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 8.4+ MB


In [26]:
(df == "").sum()

user_id             0
book_id             0
review_id           0
rating              0
review_text        32
date_added          0
date_updated        0
read_at         17363
started_at      49587
n_votes             0
n_comments          0
dtype: int64

In [27]:
df.loc[1, "review_text"]

'This is a special book. It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind. This is what I love about good science fiction - it pushes your thinking about where things can go. \n It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I\'ve read. For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc. \n It is a book about science, and aliens. The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell. Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though. \n But what would happen if our SETI stations received a message - if we found someone was out there - and the person monitoring and answering the signal on our side was 

In [28]:
df[["rating", "n_votes", "n_comments"]].astype(np.int64).describe()

,rating,n_votes,n_comments
count,100000.000000,100000.00000,100000.000000
mean,3.687550,0.96629,0.234800
std,1.226303,6.66289,1.579764
min,0.000000,-1.00000,0.000000
25%,3.000000,0.00000,0.000000
50%,4.000000,0.00000,0.000000
75%,5.000000,1.00000,0.000000
max,5.000000,1283.00000,231.000000


In [29]:
(df["rating"]==0).sum()

np.int64(3196)

In [30]:
df.loc[df["n_votes"]<0].head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
68697,25a66513848dd61de28d48013e128e92,11319360,32481703e2b779215e581ababf59469d,5,"First of all, if you haven't already read Gone...",Thu Jul 14 19:21:38 -0700 2011,Mon Nov 03 14:27:34 -0800 2014,Wed Oct 19 00:00:00 -0700 2011,Sun Oct 16 00:00:00 -0700 2011,-1,0
81536,24c32602f697dfc4cc7634df4e96fa8b,13481792,1346b97af2d59cf8a584221fb836a1c1,5,I believe The Lies Have It is the third in Jil...,Sun Apr 08 11:07:47 -0700 2012,Mon Nov 03 14:32:52 -0800 2014,Sat Apr 07 00:00:00 -0700 2012,,-1,0


Vemos como hay registros con `rating` = 0, sin embargo, el rating no puede ser inferior a 1 por lo que indicaremos estos valores como `nan`. Al ser `n_votes` un número negativo un dato erróneo, suponemos y asumimos que el valor correcto es el valor absoluto. Además, vemos como tanto `n_votes` como `n_comments` no contienen cifras muy altas, por lo que, **a priori**, con 16 bits será suficiente para representarlos.

In [31]:
df["date_added"].sample()

20381    Thu Jun 14 13:09:16 -0700 2012
Name: date_added, dtype: object

In [40]:
pd.to_datetime(df["date_added"], format="%a %b %d %H:%M:%S %z %Y", utc=True).min()

Timestamp('2006-08-20 03:01:34+0000', tz='UTC')

In [41]:
pd.to_datetime(df["date_added"], format="%a %b %d %H:%M:%S %z %Y", utc=True).max()

Timestamp('2017-10-26 04:14:32+0000', tz='UTC')

In [32]:
df["review_id"].duplicated().sum()

np.int64(0)

### Transformación
1. Eliminamos columnas que no aportan información útil.
2. Corregir valores erróneos.
3. Aplicar transformación de datos.
4. Asignar valores `nan`.

In [33]:
df.drop(columns=["read_at", "started_at"], inplace=True)

Para mejorar la comprensión, hemos decido renombrar `n_votes` a `n_likes`.

In [34]:
df.rename(columns={"n_votes": "n_likes"}, inplace=True)

In [35]:
df["user_id"] = df["user_id"].str.replace(r"\s+", "", regex=True)
df["review_id"] = df["review_id"].str.replace(r"\s+", "", regex=True)
df["review_text"] = df["review_text"].str.strip()
df["date_added"] = df["date_added"].str.strip()
df["date_updated"] = df["date_updated"].str.strip()

In [36]:
rt_nan = df["review_text"] == ""
r_nan = df["rating"] == 0

In [37]:
df["user_id"] = df["user_id"].astype("string")
df["book_id"] = df["book_id"].astype(np.uint32)
df["rating"] = df["rating"].astype(np.uint8)
df["review_id"] = df["review_id"].astype("string")
df["review_text"] = df["review_text"].astype("string")
df["date_added"] = pd.to_datetime(df["date_added"], format="%a %b %d %H:%M:%S %z %Y", utc=True)
df["date_updated"] = pd.to_datetime(df["date_updated"], format="%a %b %d %H:%M:%S %z %Y", utc=True)
df["n_likes"] = df["n_likes"].astype(np.uint).abs()
df["n_comments"] = df["n_comments"].astype(np.uint16)

In [38]:
df.loc[r_nan, "rating"] = np.nan
df.loc[rt_nan, "review_text"] = np.nan
df.isna().sum()

user_id            0
book_id            0
review_id          0
rating          3196
review_text       32
date_added         0
date_updated       0
n_likes            0
n_comments         0
dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   user_id       100000 non-null  string             
 1   book_id       100000 non-null  uint32             
 2   review_id     100000 non-null  string             
 3   rating        96804 non-null   float64            
 4   review_text   99968 non-null   string             
 5   date_added    100000 non-null  datetime64[ns, UTC]
 6   date_updated  100000 non-null  datetime64[ns, UTC]
 7   n_likes       100000 non-null  uint64             
 8   n_comments    100000 non-null  uint16             
dtypes: datetime64[ns, UTC](2), float64(1), string(3), uint16(1), uint32(1), uint64(1)
memory usage: 5.9 MB
